In [4]:
import json

from stringMatcher import levenshtein_ratio_and_distance
from time import time


# Load The database into the memory...

In [5]:
data = json.load(open("dataStore/dataFinal.json", "rb"))

# Load the Global Secondary index in the memory..

In [6]:
GIS = json.load(open("dataStore/dataFinal_GIS.json", "rb"))

# Write a function to search the colleges by Names

In [7]:
def getClosestMatch(queryString):
    # Initialize the time counter...
    tic                  = time()
    
    # Find closest match of the queryString from the Global Secondary Index...
    matchRatios          = [levenshtein_ratio_and_distance(queryString, KEY) for KEY, _ in GIS.items()]
    bestMatchRatio       = max(matchRatios)

    # If the best match ratio is less than 50% then we will assume that the records doesn't exist...
    if (bestMatchRatio < 0.5):
        return {
            "response code" : 404,
            "search time"   : f"{time()-tic} seconds",
            "message"       : "error",
            "response body" : "no match found"
        }
    
   # Otherwise...
    bestMatchRatio_index = matchRatios.index(bestMatchRatio)

    # Find the college_id of the college which is the best match to the queryString...
    bestMatch_College_Name = list(GIS)[bestMatchRatio_index]
    bestMatch_college_id   = GIS[bestMatch_College_Name]

    # Now we have college_id which is the primary key in our main data-store...
    # Retrieve the details of the college from the college_id
    collegeInfo            = data[str(bestMatch_college_id)]
    
    # Mark the search completion time...
    toc                  = time()

    # Add College_Name to the response...
    collegeInfo["College_Name"] = bestMatch_College_Name
    # Construct the response Model...
    responseModel = {
        "response code" : 200,
        "search time"   : f"{toc-tic} seconds",
        "message"       : "success",
        "response body" : collegeInfo
    }
    
    return responseModel

# Write a function to make a request and visualize the data returned by the search function...

In [8]:
def request(collegeToSearch=""):
    # college name should always be in a string...
    collegeToSearch = str(collegeToSearch)
    # Check if the college name is valid...
    if(collegeToSearch.replace(" ", "").strip() == ""):
        return f"Invalid college Name"
    
    # Make a request to fetch the college info...
    response = getClosestMatch(queryString = collegeToSearch)

    # Check the response code...
    if response["response code"] == 404:
        return f"ERROR: No records found"
    
    # otherwise...
    # Construct user ratings table...
    Rating = response["response body"]["user_rating"]
    tag    = response["response body"]["tag"]

    for rating in Rating:
        try    : tag_by_a_single_user_to_a_single_college = ", ".join([tag for tag in tags if Rating["userId"] == tag["userId"]]) 
        except : tag_by_a_single_user_to_a_single_college = ""
        break
    # Now we need to check if a user has given 
    data_to_print = f"\
    College_Name          : {response['response body']['College_Name']}\n\
    Stream         : {response['response body']['Streams']}\n\
    State          : {response['response body']['States']}\n\
    "
    print(data_to_print)


In [9]:
request(collegeToSearch="Indian Institute of Technology Madras ")

    College_Name          : Indian Institute of Technology Madras 
    Stream         : Engineering
    State          : Tamil nadu
    


In [10]:
request(collegeToSearch = "GLA University, Mathura")

    College_Name          : GLA University, Mathura 
    Stream         : Pharmacy
    State          : Uttar pradesh
    
